In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import date, datetime
import time
import re
import os
from IPython.display import clear_output
import sys

driver = webdriver.Firefox()
base_page = 'https://r*******.**'
driver.maximize_window()

In [ ]:
start_period = date.fromisoformat('2022-01-01') # year-month-day

In [ ]:
def flatten(L: list) -> list:
    if len(L) > sys.getrecursionlimit():
        sys.setrecursionlimit(len(L)*10)
    if L == []:
        return L
    if isinstance(L[0], list):
        return flatten(L[0]) + flatten(L[1:])
    rez = list(L[:1] + flatten(L[1:]))
    return rez

PAUSE_TIME = 6

In [ ]:
base_page = 'https://r********.**/ranges'
driver.get(base_page)
time.sleep(PAUSE_TIME)

link_week = [e.get_attribute('href') for e in driver.find_elements(By.CLASS_NAME,'ml-4 > a')]
link_week = list(dict.fromkeys(link_week)) # removing duplicate week-links
start_date = [datetime.strptime(link[link.find('=') + 1:link.find('&')], '%d-%m-%y').date() for link in link_week]
end_date = [datetime.strptime(link[link.find('end=') + 4:], '%d-%m-%y').date() for link in link_week]

for date in zip(start_date, end_date):
    if date[0] > start_period or date[0] <= start_period <= date[1]:
        parse_list = link_week[start_date.index(date[0]):]
        break
    else:
        parse_list = []

In [ ]:
type_activity_final = []
dates_final = []
title_final = []
detail_links_final = []
source_links_final = []
content_final = []

for link in parse_list:
    driver.get(link)
    time.sleep(PAUSE_TIME)
    type_activity = [elem.text for elem in driver.find_elements(By.TAG_NAME,'article > div > div > p')]
    dates = [datetime.strptime(str(elem.text).split()[0] + '.' + str(datetime.now().date().year), '%d.%m.%Y').date() 
             for elem in driver.find_elements(By.TAG_NAME,'article > div > p')]
    title = [elem.text for elem in driver.find_elements(By.TAG_NAME,'article > h1')]

    detail_links = []
    source_links = []
    temp_list=[]
    for elem in driver.find_elements(By.TAG_NAME,'body > div > div > main > section > article'):
        for sub_elem in elem.find_elements(By.TAG_NAME,'div > a'):
            temp_list.append(sub_elem.get_attribute('href'))
        if len(temp_list) == 1:
            temp_list.append('Нет источника')
        detail_links.append(temp_list[0])
        source_links.append(temp_list[1])
        temp_list=[]

    k = sum([True for date in dates if date >= start_period])
    
    for det_link in detail_links[:k]:
        driver.get(det_link)
        time.sleep(PAUSE_TIME)
        content_final.append([elem.text for elem in driver.find_elements(By.TAG_NAME,'body > div > div > div > div > article > div > p')][:-2])
    
    
    type_activity_final.extend(type_activity[:k])
    dates_final.extend(dates[:k])
    title_final.extend(title[:k])
    detail_links_final.extend(detail_links[:k])
    source_links_final.extend(source_links[:k])
    
    
    assert(len(type_activity) == len(dates) == len(title) == len(detail_links) == len(source_links)), 'Assert Error'
    assert(len(type_activity_final) == len(dates_final) == len(title_final) == len(detail_links_final) \
           == len(source_links_final) == len(content_final)), 'Assert Error'
    
    clear_output(wait=True)
    print(len(type_activity_final), len(dates_final), len(title_final), len(detail_links_final), \
          len(source_links_final), len(content_final))

In [ ]:
df = pd.DataFrame()

df['Тип события'] = type_activity_final
df['Дата события'] = dates_final
df['Название'] = title_final
df['Ссылка'] = detail_links_final
df['Источник'] = source_links_final
df['Контент'] = content_final

for idx,_ in df.iterrows():
    if type(df.iloc[idx]['Контент']) is list:
        df.iloc[idx]['Контент'] = ', '.join(flatten(df.iloc[idx]['Контент']))
df

In [ ]:
file_name = base_page.split('/')[2].split('.')[0] + '_' + str(datetime.now().date()) + '.xlsx'
with pd.ExcelWriter(file_name) as writer:
    df.to_excel(writer, sheet_name='Лист1', index=False)